# 🔍 Perguntas:

1. 💰 Qual foi o total de receita gerada no período analisado?
2. 🧾 Qual foi o ticket médio por pedido (valor médio gasto por order_id)?
3. 🍕 Quais são as pizzas mais vendidas em quantidade?
4. 💸 Quais pizzas geraram mais receita total?
5. 📆 Em quais datas houve os maiores volumes de vendas?
6. 📊 Quais tamanhos de pizza foram mais vendidos?
7. 🏷️ Qual categoria de pizza (ex: clássica, vegetariana) gerou mais receita?
8. ⏰ Em quais horários do dia ocorrem mais pedidos?
9. 🧂 Quais são os ingredientes mais comuns nas pizzas mais populares?
10. 🧮 Qual é a média de pizzas vendidas por pedido?


# ✅ 1. Carregar o CSV no Pandas

In [3]:
import pandas as pd

# Ler o arquivo CSV
df = pd.read_csv("pizza_sales.csv")

# Ver as primeiras linhas
df.head(5)

,pizza_id,order_id,pizza_name_id,quantity,order_date,order_time,unit_price,total_price,pizza_size,pizza_category,pizza_ingredients,pizza_name
0,1.0,1.0,hawaiian_m,1.0,1/1/2015,11:38:36,13.25,13.25,M,Classic,"Sliced Ham, Pineapple, Mozzarella Cheese",The Hawaiian Pizza
1,2.0,2.0,classic_dlx_m,1.0,1/1/2015,11:57:40,16.00,16.00,M,Classic,"Pepperoni, Mushrooms, Red Onions, Red Peppers,...",The Classic Deluxe Pizza
2,3.0,2.0,five_cheese_l,1.0,1/1/2015,11:57:40,18.50,18.50,L,Veggie,"Mozzarella Cheese, Provolone Cheese, Smoked Go...",The Five Cheese Pizza
3,4.0,2.0,ital_supr_l,1.0,1/1/2015,11:57:40,20.75,20.75,L,Supreme,"Calabrese Salami, Capocollo, Tomatoes, Red Oni...",The Italian Supreme Pizza
4,5.0,2.0,mexicana_m,1.0,1/1/2015,11:57:40,16.00,16.00,M,Veggie,"Tomatoes, Red Peppers, Jalapeno Peppers, Red O...",The Mexicana Pizza


# ✅ 2. Criar banco SQLite e carregar o DataFrame

In [5]:
import sqlite3

# Criar conexão com banco em memória (ou use um arquivo .db se quiser salvar)
conn = sqlite3.connect(":memory:")

# Enviar o DataFrame para o SQLite
df.to_sql("pizza_sales", conn, index=False, if_exists="replace")


48620

In [7]:
# Exemplo: total de vendas por tipo de pizza
query = """
SELECT * 
FROM pizza_sales
LIMIT 5;
"""

df_resultado = pd.read_sql_query(query, conn)
df_resultado

,pizza_id,order_id,pizza_name_id,quantity,order_date,order_time,unit_price,total_price,pizza_size,pizza_category,pizza_ingredients,pizza_name
0,1.0,1.0,hawaiian_m,1.0,1/1/2015,11:38:36,13.25,13.25,M,Classic,"Sliced Ham, Pineapple, Mozzarella Cheese",The Hawaiian Pizza
1,2.0,2.0,classic_dlx_m,1.0,1/1/2015,11:57:40,16.00,16.00,M,Classic,"Pepperoni, Mushrooms, Red Onions, Red Peppers,...",The Classic Deluxe Pizza
2,3.0,2.0,five_cheese_l,1.0,1/1/2015,11:57:40,18.50,18.50,L,Veggie,"Mozzarella Cheese, Provolone Cheese, Smoked Go...",The Five Cheese Pizza
3,4.0,2.0,ital_supr_l,1.0,1/1/2015,11:57:40,20.75,20.75,L,Supreme,"Calabrese Salami, Capocollo, Tomatoes, Red Oni...",The Italian Supreme Pizza
4,5.0,2.0,mexicana_m,1.0,1/1/2015,11:57:40,16.00,16.00,M,Veggie,"Tomatoes, Red Peppers, Jalapeno Peppers, Red O...",The Mexicana Pizza


In [8]:
print(df.columns)

Index(['pizza_id', 'order_id', 'pizza_name_id', 'quantity', 'order_date',
       'order_time', 'unit_price', 'total_price', 'pizza_size',
       'pizza_category', 'pizza_ingredients', 'pizza_name'],
      dtype='object')


# 📊 EDA com SQL e Python!

## -> Qual foi o total de receita gerada no período analisado?

In [16]:

query = """
SELECT SUM(total_price) AS total_vendas 
FROM pizza_sales;
"""

df_resultado = pd.read_sql_query(query, conn)
df_resultado

,total_vendas
0,817860.05


## -> Qual foi o ticket médio por pedido (valor médio gasto por order_id)?

In [20]:
query = """
SELECT SUM(total_price) / COUNT(DISTINCT order_id) AS ticket_medio
FROM pizza_sales;
"""

df_resultado = pd.read_sql_query(query, conn)
df_resultado

,ticket_medio
0,38.307262


## -> Qual foi o ticket médio por pedido (valor médio gasto por order_id)?

In [21]:
query = """
SELECT SUM(total_price) AS total_vendas, SUM(total_price) / COUNT(DISTINCT order_id) AS ticket_medio
FROM pizza_sales;
"""

df_resultado = pd.read_sql_query(query, conn)
df_resultado

,total_vendas,ticket_medio
0,817860.05,38.307262


## -> Quais são as pizzas mais vendidas em quantidade?

In [32]:
query = """
SELECT DISTINCT pizza_name_id, COUNT(order_id) as quantidade_pedidos
FROM pizza_sales
GROUP BY pizza_name_id
ORDER BY quantidade_pedidos DESC;
"""

df_resultado = pd.read_sql_query(query, conn)
df_resultado

,pizza_name_id,quantidade_pedidos
0,big_meat_s,1811
1,thai_ckn_l,1365
2,five_cheese_l,1359
3,four_cheese_l,1273
4,classic_dlx_m,1159
...,...,...
86,mexicana_s,160
87,calabrese_s,99
88,ckn_alfredo_s,96
89,green_garden_l,94


## -> Quais pizzas geraram mais receita total?

In [41]:

query = """
SELECT DISTINCT pizza_name, SUM(total_price) AS total_vendas 
FROM pizza_sales
GROUP BY pizza_name
ORDER BY total_vendas DESC
LIMIT 10;
"""

df_resultado = pd.read_sql_query(query, conn)
df_resultado

,pizza_name,total_vendas
0,The Thai Chicken Pizza,43434.25
1,The Barbecue Chicken Pizza,42768.00
2,The California Chicken Pizza,41409.50
3,The Classic Deluxe Pizza,38180.50
4,The Spicy Italian Pizza,34831.25
5,The Southwest Chicken Pizza,34705.75
6,The Italian Supreme Pizza,33476.75
7,The Hawaiian Pizza,32273.25
8,The Four Cheese Pizza,32265.70
9,The Sicilian Pizza,30940.50


In [44]:
query = """
SELECT pizza_name, pizza_size, SUM(total_price) AS receita
FROM pizza_sales
GROUP BY pizza_name
ORDER BY receita DESC
LIMIT 10;
"""

df_resultado = pd.read_sql_query(query, conn)
df_resultado

,pizza_name,pizza_size,receita
0,The Thai Chicken Pizza,L,43434.25
1,The Barbecue Chicken Pizza,S,42768.00
2,The California Chicken Pizza,L,41409.50
3,The Classic Deluxe Pizza,M,38180.50
4,The Spicy Italian Pizza,L,34831.25
5,The Southwest Chicken Pizza,L,34705.75
6,The Italian Supreme Pizza,L,33476.75
7,The Hawaiian Pizza,M,32273.25
8,The Four Cheese Pizza,L,32265.70
9,The Sicilian Pizza,L,30940.50


## -> Quais tamanhos de pizza foram mais vendidos?

In [51]:
query = """
SELECT pizza_size, SUM(total_price) AS receita
FROM pizza_sales
GROUP BY pizza_size
ORDER BY receita DESC
LIMIT 10;
"""

df_resultado = pd.read_sql_query(query, conn)
df_resultado

,pizza_size,receita
0,L,375318.70
1,M,249382.25
2,S,178076.50
3,XL,14076.00
4,XXL,1006.60


## -> Quais tamanhos de pizza foram mais vendidos?

In [48]:
≈query = """
SELECT pizza_category, SUM(total_price) AS receita
FROM pizza_sales
GROUP BY pizza_category
ORDER BY receita DESC
LIMIT 10;
"""

df_resultado = pd.read_sql_query(query, conn)
df_resultado

,pizza_category,receita
0,Classic,220053.10
1,Supreme,208197.00
2,Chicken,195919.50
3,Veggie,193690.45


## -> Em quais horários do dia ocorrem mais pedidos?

In [54]:

query = """
SELECT order_time, COUNT(order_id) as quantidade_pedidos
FROM pizza_sales
GROUP BY order_time
ORDER BY quantidade_pedidos DESC
LIMIT 10;
"""

df_resultado = pd.read_sql_query(query, conn)
df_resultado

,order_time,quantidade_pedidos
0,12:32:00,26
1,11:59:10,24
2,12:53:29,23
3,12:52:36,23
4,12:26:04,23
5,13:31:27,21
6,12:25:12,21
7,11:57:15,21
8,11:50:01,21
9,13:58:37,20


## -> Quais são os ingredientes mais comuns nas pizzas mais populares?


In [55]:
query = """
SELECT pizza_ingredients, COUNT(order_id) as quantidade_pedidos
FROM pizza_sales
GROUP BY pizza_ingredients
ORDER BY quantidade_pedidos DESC
LIMIT 10;
"""

df_resultado = pd.read_sql_query(query, conn)
df_resultado

,pizza_ingredients,quantidade_pedidos
0,"Pepperoni, Mushrooms, Red Onions, Red Peppers,...",2416
1,"Barbecued Chicken, Red Peppers, Green Peppers,...",2372
2,"Sliced Ham, Pineapple, Mozzarella Cheese",2370
3,"Mozzarella Cheese, Pepperoni",2369
4,"Chicken, Pineapple, Tomatoes, Red Peppers, Tha...",2315
5,"Chicken, Artichoke, Spinach, Garlic, Jalapeno ...",2302
6,"Coarse Sicilian Salami, Tomatoes, Green Olives...",1887
7,"Capocollo, Tomatoes, Goat Cheese, Artichokes, ...",1887
8,"Chicken, Tomatoes, Red Peppers, Red Onions, Ja...",1885
9,"Ricotta Cheese, Gorgonzola Piccante Cheese, Mo...",1850


## -> Qual é a média de pizzas vendidas por pedido?


In [63]:
query = """
SELECT 
SUM(quantity) * 1.0 / COUNT(DISTINCT order_id) AS media_pizzas_por_pedido
FROM pizza_sales;

"""

df_resultado = pd.read_sql_query(query, conn)
df_resultado

,media_pizzas_por_pedido
0,2.321967
